<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [85]:
import requests
from lxml import html
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [86]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content, 'html.parser')
data = []
table = soup.find('table', class_="wikitable")
table_body = table.find('tbody')
rows = table_body.find_all('tr')

for row in rows:    
    cols = row.find_all(['td','th'])
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

Ignore cells with a borough that is Not assigned.

In [87]:
df = pd.DataFrame(data)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df=df[~df.Borough.str.contains("Not assigned")]
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Now we will combine all those neighbourhood(by a comma) whose postcode is same.

In [89]:
gb = df.groupby(('Postcode','Borough'))
result = gb['Neighbourhood'].agg([('Neighbourhood', ', '.join)])
result=result.reset_index()
result.head()
result['Neighbourhood'] = np.where(result['Neighbourhood']=='Not assigned', result['Borough'], result['Neighbourhood'])
result.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Getting the shape of the resultant dataframe.

In [90]:
result.shape

(103, 3)